## Import Libraries

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

pd.set_option('future.no_silent_downcasting', True)

## Read & Understand Data

In [ ]:
# Reading the dataset
df = pd.read_csv('Sources/car_prices.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

## Data Cleaning

In [ ]:
#Renaming Columns
df.rename(columns={'make': 'brand'}, inplace=True)

In [ ]:
# Converting 'saledate' to datetime format
df.saledate = df.saledate.apply(lambda x: str(x)[:-6])
df.saledate = pd.to_datetime(df.saledate, format='%a %b %d %Y %H:%M:%S GMT%z', utc=True, errors='coerce')

In [ ]:
# Extracting day and month from 'saledate'
df['day'] = df['saledate'].dt.day_name()
df['month'] = df['saledate'].dt.month_name()

In [ ]:
df.info()

In [ ]:
# Dropping unnecessary columns
df.drop(['mmr', 'vin', 'trim'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# Checking unique values in certain columns
df.year.unique()

In [ ]:
df.brand.unique()  # Problem with capitalization

In [ ]:
df.model.unique()  # Problem with capitalization

In [ ]:
df.body.unique()  # Problem with capitalization

In [ ]:
df.transmission.unique()  # Null values + "sedan"

In [ ]:
df.state.unique()  # Wrong Values

In [ ]:
df.color.unique()  # Wrong data + '-'

In [ ]:
df.interior.unique()  # Wrong data + '-'

In [ ]:
df.seller.unique()

In [ ]:
# Handling capitalization issues in object columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.title()
    print(col, df[col].unique(), "\n\n")

In [ ]:
# Dropping rows with incorrect 'transmission' values
df[df.transmission.isin(["sedan", "Sedan"])]

In [ ]:
df.drop(df[df.transmission.isin(["sedan", "Sedan"])].index, inplace=True, axis=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.transmission.unique()

In [ ]:
# Handling erroneous entries in 'state' column
df[df.state.str.len() > 2]

In [ ]:
# Handling numeric values in 'color' and 'interior' columns
df[df.color.apply(lambda x: str(x).isnumeric() if pd.notnull(x) else False)]
#also fixed

In [ ]:
#handling numeric values in interior
df[df.interior.apply(lambda x: str(x).isnumeric() if pd.notnull(x) else False)]
#also fixed

In [ ]:
df.head()

In [ ]:
# Mapping State abbreviations to their full names
abbrev_to_us_State = {
    "al": "Alabama",
    "ak": "Alaska",
    "az": "Arizona",
    "ar": "Arkansas",
    "ca": "California",
    "co": "Colorado",
    "ct": "Connecticut",
    "de": "Delaware",
    "fl": "Florida",
    "ga": "Georgia",
    "hi": "Hawaii",
    "id": "Idaho",
    "il": "Illinois",
    "in": "Indiana",
    "ia": "Iowa",
    "ks": "Kansas",
    "ky": "Kentucky",
    "la": "Louisiana",
    "me": "Maine",
    "md": "Maryland",
    "ma": "Massachusetts",
    "mi": "Michigan",
    "mn": "Minnesota",
    "ms": "Mississippi",
    "mo": "Missouri",
    "mt": "Montana",
    "ne": "Nebraska",
    "nv": "Nevada",
    "nh": "New Hampshire",
    "nj": "New Jersey",
    "nm": "New Mexico",
    "ny": "New York",
    "nc": "North Carolina",
    "nd": "North Dakota",
    "oh": "Ohio",
    "ok": "Oklahoma",
    "or": "Oregon",
    "pa": "Pennsylvania",
    "ri": "Rhode Island",
    "sc": "South Carolina",
    "sd": "South Dakota",
    "tn": "Tennessee",
    "tx": "Texas",
    "ut": "Utah",
    "vt": "Vermont",
    "va": "Virginia",
    "wa": "Washington",
    "wv": "West Virginia",
    "wi": "Wisconsin",
    "wy": "Wyoming",
    "dc": "District of Columbia",
    "as": "American Samoa",
    "gu": "Guam",
    "mp": "Northern Mariana Islands",
    "pr": "Puerto Rico",
    "um": "United States Minor Outlying Islands",
    "vi": "U.S. Virgin Islands",
}
df['state'] = df['state'].str.lower().str.strip()
df['state'] = df['state'].map(abbrev_to_us_State)

## Handling Outliers

In [ ]:
df.describe()

In [ ]:
# Handling outliers in 'odometer' column
px.box(df, x=df.odometer)

In [ ]:
df.drop(df[df.odometer >= 400000].index, inplace=True, axis=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
# Handling outliers in 'sellingprice' column
px.box(df, x=df.sellingprice)

In [ ]:
df[df.sellingprice > 150000]

In [ ]:
df.describe()

## Handling Missing Data

In [ ]:
# Handling missing data in 'color' and 'interior' columns using mode imputation
df[df.color == '—'].shape[0]

In [ ]:
df[df.interior == '—'].shape[0]

In [ ]:
df[['color', 'interior']] = df[['color', 'interior']].replace('—', np.nan)

In [ ]:
# Replacing with mode
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

In [ ]:
df[['color', 'interior']] = imputer.fit_transform(df[['color', 'interior']])
df.color.unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
df.columns

In [ ]:
# Dropping rows from columns with less than 2% null values
df.dropna(subset=['year', 'brand', 'model', 'state',
                  'odometer', 'color', 'interior', 'seller', 'sellingprice', 'saledate'], inplace=True, axis=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
# Dropping from 'body' column as well
df.dropna(subset=['body'], inplace=True, axis=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
# Handling missing values in 'condition' column using KNN imputation
px.histogram(df, x='condition', nbins=10)

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)

In [ ]:
df.condition = imputer.fit_transform(df[['condition']])

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
px.histogram(df, x='condition', nbins=10)

In [ ]:
# Handling missing values in 'transmission' column based on other information
px.pie(df, names='transmission')

In [ ]:
df.columns

In [ ]:
df['transmission'] = df.groupby(['year', 'brand', 'model'])['transmission'].transform(lambda x: x.fillna(x.iloc[0]))

In [ ]:
px.pie(df, names='transmission')

In [ ]:
df.dropna(subset=['transmission'], inplace=True, axis=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Saving the cleaned dataset to a new CSV file
df.to_csv('Sources/car_prices_cleaned.csv')

# Analysis

## Top Car Brands and Models Analysis

#### Most sold cars and Models

In [ ]:
# Plotting histogram for top 25 car Brands
top_brand = df['brand'].value_counts().nlargest(25).index
top_brand = df[df['brand'].isin(top_brand)].reset_index(drop=True)

In [ ]:
fig = px.histogram(top_brand, x='brand', color='brand')
fig.update_layout(
    title='Cars Sold For Each Brand',
    xaxis_title='brand',
    yaxis_title='Cars Sold',
    title_x=0.5
)
fig.show()

In [ ]:
# Plotting histogram for top 25 car Models
top_model = df['model'].value_counts().nlargest(25).index
top_model = df[df['model'].isin(top_model)].reset_index(drop=True)

In [ ]:
fig = px.histogram(top_model, x='model', color='model')
fig.update_layout(
    title='Cars Sold For Each Model',
    xaxis_title='Model',
    yaxis_title='Cars Sold',
    title_x=0.5,
)
fig.show()

#### Favourite Body types

In [ ]:
# Plotting pie chart for top 4 Body types
top_body = df['body'].value_counts().nlargest(4).index
top_body = df[df['body'].isin(top_body)].reset_index(drop=True)
body_counts = top_body['body'].value_counts()

In [ ]:
fig = px.pie(top_body, values=body_counts.values, names=body_counts.index)
fig.update_layout(
    title='Cars Sold For Each Body Type',
    title_x=0.5
)
fig.show()

#### Favourite Color

In [ ]:
#Mapping Colors
available_color = ["aliceblue", "antiquewhite", "aqua", "aquamarine", "azure",
                   "beige", "bisque", "black", "blanchedalmond", "blue",
                   "blueviolet", "brown", "burlywood", "cadetblue",
                   "chartreuse", "chocolate", "coral", "cornflowerblue",
                   "cornsilk", "crimson", "cyan", "darkblue", "darkcyan",
                   "darkgoldenrod", "darkgray", "darkgrey", "darkgreen",
                   "darkkhaki", "darkmagenta", "darkolivegreen", "darkorange",
                   "darkorchid", "darkred", "darksalmon", "darkseagreen",
                   "darkslateblue", "darkslategray", "darkslategrey",
                   "darkturquoise", "darkviolet", "deeppink", "deepskyblue",
                   "dimgray", "dimgrey", "dodgerblue", "firebrick",
                   "floralwhite", "forestgreen", "fuchsia", "gainsboro",
                   "ghostwhite", "gold", "goldenrod", "gray", "grey", "green",
                   "greenyellow", "honeydew", "hotpink", "indianred", "indigo",
                   "ivory", "khaki", "lavender", "lavenderblush", "lawngreen",
                   "lemonchiffon", "lightblue", "lightcoral", "lightcyan",
                   "lightgoldenrodyellow", "lightgray", "lightgrey",
                   "lightgreen", "lightpink", "lightsalmon", "lightseagreen",
                   "lightskyblue", "lightslategray", "lightslategrey",
                   "lightsteelblue", "lightyellow", "lime", "limegreen",
                   "linen", "magenta", "maroon", "mediumaquamarine",
                   "mediumblue", "mediumorchid", "mediumpurple",
                   "mediumseagreen", "mediumslateblue", "mediumspringgreen",
                   "mediumturquoise", "mediumvioletred", "midnightblue",
                   "mintcream", "mistyrose", "moccasin", "navajowhite", "navy",
                   "oldlace", "olive", "olivedrab", "orange", "orangered",
                   "orchid", "palegoldenrod", "palegreen", "paleturquoise",
                   "palevioletred", "papayawhip", "peachpuff", "peru", "pink",
                   "plum", "powderblue", "purple", "red", "rosybrown",
                   "royalblue", "rebeccapurple", "saddlebrown", "salmon",
                   "sandybrown", "seagreen", "seashell", "sienna", "silver",
                   "skyblue", "slateblue", "slategray", "slategrey", "snow",
                   "springgreen", "steelblue", "tan", "teal", "thistle", "tomato",
                   "turquoise", "violet", "wheat", "white", "whitesmoke",
                   "yellow", "yellowgreen"
                   ]

In [ ]:
color_map = {'Burgundy': 'darkred', 'Charcoal': 'olive', 'Off-White': 'whitesmoke'}
unique_colors = df['color'].unique()
for color in unique_colors:
    if color.lower() in available_color:
        color_map[color] = color.lower()

In [ ]:
# Plotting pie chart for top 4 Colors
top_color = df['color'].value_counts().nlargest(4).index
top_color = df[df['color'].isin(top_color)].reset_index(drop=True)
color_counts = top_color['color'].value_counts()

In [ ]:
fig = px.pie(top_color, values=color_counts.values, names=color_counts.index, color=color_counts.index,
             color_discrete_map=color_map)
fig.update_layout(
    title='Cars Sold For Each Color Type',
    title_x=0.5
)
fig.show()

In [ ]:
# Plotting pie chart for top 4 interior Colors
top_interior = df['interior'].value_counts().nlargest(4).index
top_interior = df[df['interior'].isin(top_interior)].reset_index(drop=True)
interior_counts = top_interior['interior'].value_counts()

In [ ]:
fig = px.pie(top_interior, values=interior_counts.values, names=interior_counts.index, color=interior_counts.index,
             color_discrete_map=color_map)
fig.update_layout(
    title='Cars Sold For Each interior Type',
    title_x=0.5
)
fig.show()

## Total Sales Analysis

#### Top Brands and Model by sales

In [ ]:
# Calculating total sales by car Brand
total_sales_brand = df.groupby('brand')['sellingprice'].sum().reset_index()
total_sales_brand = total_sales_brand.nlargest(20, 'sellingprice')

In [ ]:
fig = px.bar(total_sales_brand, x='brand', y='sellingprice',
             title='Total Sales by Car Brand (Top 20)',
             labels={'sellingprice': 'Total Sales', 'brand': 'Car Brand'},
             color='brand')
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
# Calculating total sales by car Model
total_sales_model = df.groupby('model')['sellingprice'].sum().reset_index()
total_sales_model = total_sales_model.nlargest(25, 'sellingprice')

In [ ]:
fig = px.bar(total_sales_model, x='model', y='sellingprice',
             title='Total Sales by Car Model',
             labels={'sellingprice': 'Total Sales', 'model': 'Car Model'},
             color='model')
fig.update_layout(title_x=0.5)
fig.show()

#### Days and months distribution

In [ ]:
# Plotting pie chart for number of cars sold on each day
daily_sales = df.groupby('day').size().reset_index(name='count')

In [ ]:
fig = px.pie(daily_sales, values='count', names='day',
             title='Number of Cars Sold on Each Day', hole=0.4)
fig.show()

In [ ]:
# Plotting bar chart for number of cars sold on each month
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October',
               'November', 'December']
monthly_sales = df.groupby('month').size().reset_index(name='count')
monthly_sales['month'] = pd.Categorical(monthly_sales['month'], categories=month_order, ordered=True)
monthly_sales = monthly_sales.sort_values('month')

In [ ]:
fig = px.bar(monthly_sales, y='count', x='month',
             title='Number of Cars Sold on Each Month',
             labels={'month': 'Month', 'count': 'Cars Sold'},
             color='month')
fig.show()

## Price Analysis

#### What is the average selling price for each car Brand and Model

In [ ]:
# Calculating average selling price by car Brand
avg_price_brand = df.groupby('brand')['sellingprice'].mean().reset_index()
avg_price_brand = avg_price_brand.sort_values('sellingprice', ascending=False).head(5)

In [ ]:
fig = px.bar(avg_price_brand, x='brand', y='sellingprice',
             title='Average Selling Price by Brand',
             labels={'sellingprice': 'Average Selling Price', 'brand': 'Brand'},
             color='brand')
fig.show()

In [ ]:
# Calculating average selling price by car Model
avg_price_model = df.groupby('model')['sellingprice'].mean().reset_index()
avg_price_model = avg_price_model.sort_values('sellingprice', ascending=False).head(5)

In [ ]:
fig = px.bar(avg_price_model, x='model', y='sellingprice',
             title='Average Selling Price by Model',
             labels={'sellingprice': 'Average Selling Price', 'model': 'Model'},
             color='model')
fig.show()

#### relation between Odometer and selling price

In [ ]:
# Plotting scatter plot to visualize the correlation between Odometer readings and selling prices
fig = px.scatter(df, x='odometer', y='sellingprice',
                 title='Correlation between Odometer Readings and Selling Prices',
                 labels={'odometer': 'Odometer (miles)', 'sellingprice': 'Selling Price'})
fig.show()

In [ ]:
# Calculating average selling price by condition
average_selling_price_by_condition = df.groupby('condition')['sellingprice'].mean().reset_index()

# Plotting line graph to visualize the relationship between condition and average selling price
fig = px.line(average_selling_price_by_condition, x='condition', y='sellingprice',
              title='Relation between Condition and Average Selling Price',
              labels={'condition': 'Condition', 'sellingprice': 'Average Selling Price'})
fig.show()

#### relation between Color and selling price

In [ ]:
# Calculating average selling price by Color
average_selling_price_by_color = df.groupby('color')['sellingprice'].mean().reset_index()
average_selling_price_by_color = average_selling_price_by_color.sort_values('sellingprice', ascending=False).head(20)

fig = px.bar(average_selling_price_by_color,
             x='color',
             y='sellingprice',
             title='Top 20 Colors by Average Selling Price',
             labels={'color': 'Color', 'sellingprice': 'Average Selling Price'},
             color='color',
             color_discrete_map=color_map)
fig.show()


In [ ]:
# Calculating average selling price by interior Color
average_selling_price_by_interior = df.groupby('interior')['sellingprice'].mean().reset_index()
average_selling_price_by_interior = average_selling_price_by_interior.sort_values('sellingprice', ascending=False).head(
    20)

fig = px.bar(average_selling_price_by_interior, x='interior', y='sellingprice',
             title='Average Selling Price by interior',
             labels={'interior': 'interior', 'sellingprice': 'Average Selling Price'},
             color='interior',
             color_discrete_map=color_map)

fig.show()

## Sellers Analysis

#### Top sellers

In [ ]:
# Calculating total number of vehicles sold by each seller
vehicles_sold_by_seller = df['seller'].value_counts().reset_index().head(10)
vehicles_sold_by_seller.columns = ['seller', 'vehicles_sold']

fig = px.bar(vehicles_sold_by_seller, x='seller', y='vehicles_sold',
             title='Total Number of Vehicles Sold by Each Seller',
             color='seller')
fig.update_layout(xaxis_title='Seller', yaxis_title='Total Number of Vehicles Sold')
fig.show()

#### Total Sales By seller

In [ ]:
# Calculating total sales by each seller
top_sellers_sales = df.groupby('seller')['sellingprice'].sum().reset_index()
top_sellers_sales = top_sellers_sales.nlargest(10, 'sellingprice')

fig = px.bar(top_sellers_sales, x='seller', y='sellingprice',
             title='Top 10 Sellers - Total Sales',
             color='sellingprice')
fig.update_layout(xaxis_title='Seller', yaxis_title='Total Sales ($)')

fig.show()

#### Avg odometer for each seller

In [ ]:
# Calculating average odometer for each seller
average_odometer_by_seller = df.groupby('seller')['odometer'].mean().reset_index()
average_odometer_by_seller = average_odometer_by_seller.nlargest(5, 'odometer')
fig = px.bar(average_odometer_by_seller, x='seller', y='odometer',
             title='Average Vehicle Odometer by Seller',
             labels={'seller': 'Seller', 'odometer': 'Average odometer', },
             color='odometer',
             color_continuous_scale='ice')

fig.show()

## States Analysis

#### Sales for each State

In [ ]:
# Calculating total sales revenue by state
top_states_by_revenue = df.groupby('state')['sellingprice'].sum().reset_index()
top_states_by_revenue = top_states_by_revenue.sort_values(by='sellingprice', ascending=False)

fig = px.bar(top_states_by_revenue.head(10), x='state', y='sellingprice',
             title='Top states by Total Sales Revenue',
             labels={'state': 'State', 'sellingprice': 'Total Sales Revenue'},
             color='state')

fig.show()

#### avg condition for each state

In [ ]:
# Calculating average condition for each state
top_states_con = df.groupby('state')['condition'].mean().reset_index().head(5)

top_states_con = top_states_con.sort_values('condition', ascending=False)

fig = px.bar(top_states_con, x='state', y='condition',
             title='Top 5 States with Highest Average Vehicle Condition',
             labels={'state': 'State', 'condition': 'Average Condition'},
             color='state')

fig.show()

#### avg price for each state

In [ ]:
# Calculating average selling price of vehicles by state
top_states_avg_price = df.groupby('state')['sellingprice'].mean().reset_index()
top_states_avg_price = top_states_avg_price.sort_values(by='sellingprice', ascending=False).head(5)

fig = px.bar(top_states_avg_price, x='state', y='sellingprice',
             title='Average Selling Price of Vehicles by State',
             labels={'state': 'State', 'sellingprice': 'Average Selling Price'},
             color='state')

fig.show()